In [ ]:
!pip install cryptography


In [ ]:
from cryptography.fernet import Fernet
import json
import base64
import datetime

def generate_key():
    return Fernet.generate_key()

def encrypt_message(message, key):
    f = Fernet(key)
    encrypted = f.encrypt(message.encode())
    return encrypted

def decrypt_message(encrypted_message, key):
    f = Fernet(key)
    return f.decrypt(encrypted_message).decode()

def create_key_metadata(key_version, key):
    metadata = {
        "version": key_version,
        "created_at": datetime.datetime.utcnow().isoformat(),
        "key": key.decode()
    }
    return metadata

def rotate_key(old_key, new_key, encrypted_data):

    old_fernet = Fernet(old_key)
    plaintext = old_fernet.decrypt(encrypted_data)

    new_fernet = Fernet(new_key)
    new_encrypted_data = new_fernet.encrypt(plaintext)
    return new_encrypted_data

key_v1 = generate_key()
metadata_v1 = create_key_metadata("v1", key_v1)
print("Key V1 Metadata:\n", json.dumps(metadata_v1, indent=2))

message = "Confidential: Key management is critical!"
encrypted_msg_v1 = encrypt_message(message, key_v1)
print("\nEncrypted Message V1:\n", encrypted_msg_v1)

decrypted_msg_v1 = decrypt_message(encrypted_msg_v1, key_v1)
print("\nDecrypted Message V1:\n", decrypted_msg_v1)

key_v2 = generate_key()
metadata_v2 = create_key_metadata("v2", key_v2)
print("\nKey V2 Metadata:\n", json.dumps(metadata_v2, indent=2))

encrypted_msg_v2 = rotate_key(key_v1, key_v2, encrypted_msg_v1)
print("\nEncrypted Message V2 (after key rotation):\n", encrypted_msg_v2)

decrypted_msg_v2 = decrypt_message(encrypted_msg_v2, key_v2)
print("\nDecrypted Message V2:\n", decrypted_msg_v2)


Key V1 Metadata:
 {
  "version": "v1",
  "created_at": "2025-07-31T17:54:16.202107",
  "key": "CgZebbG1vnJ-ovjhYF3ZwXIGX2CZgLWrKh44gsDmUSk="
}

Encrypted Message V1:
 b'gAAAAABoi63IaV9bf41n9A1cWCvdxeVpgLHoIQ87ayoruM2VVhQgGVJTiFCbOOFNcgwPo5Ozj-ZBwGC1nRGzcgfI4h7me0fgvSd41ONGRTyplF6wbVnOQ7nXeAmQsukGFjTKRDdtL1Ho'

Decrypted Message V1:
 Confidential: Key management is critical!

Key V2 Metadata:
 {
  "version": "v2",
  "created_at": "2025-07-31T17:54:16.202832",
  "key": "yDs_O1gfXLpMy4KSk0qAoI8PF6InfSADnF1NW-6PktI="
}

Encrypted Message V2 (after key rotation):
 b'gAAAAABoi63IxjYPEW5BGOg0RlVRCSI48r0mnMMizGEdUCM7Ud6emhxu2SE-cTVaO9yZ4uj46noIQJfg6KwpKIqcoVQaQC81vvb3atYiTnINlrArfB0ocMiWTIV664sPdNrsJFN2I-uX'

Decrypted Message V2:
 Confidential: Key management is critical!


In [ ]:
import os
import json
import base64
import datetime
from cryptography.fernet import Fernet


In [ ]:
# Directory to store keys
os.makedirs("keyvault", exist_ok=True)

# Key vault JSON metadata file
vault_file = "keyvault/vault.json"

# Initialize key vault if it doesn't exist
if not os.path.exists(vault_file):
    with open(vault_file, "w") as f:
        json.dump({"keys": []}, f)


In [ ]:
def generate_key():
    key = Fernet.generate_key()
    return key

def save_key_with_metadata(key, version):
    timestamp = datetime.datetime.now().isoformat()
    key_filename = f"keyvault/key_v{version}.key"

    # Save key file
    with open(key_filename, "wb") as f:
        f.write(key)

    # Save metadata to vault
    with open(vault_file, "r") as f:
        data = json.load(f)

    data["keys"].append({
        "version": version,
        "filename": key_filename,
        "timestamp": timestamp
    })

    with open(vault_file, "w") as f:
        json.dump(data, f, indent=4)

    print(f"✅ Key version {version} saved with metadata.")


In [ ]:
def load_key(version):
    with open(vault_file, "r") as f:
        data = json.load(f)

    for entry in data["keys"]:
        if entry["version"] == version:
            with open(entry["filename"], "rb") as kf:
                key = kf.read()
                return Fernet(key)

    raise ValueError(f"Key version {version} not found.")


In [ ]:
def load_key(version):
    with open(vault_file, "r") as f:
        data = json.load(f)

    for entry in data["keys"]:
        if entry["version"] == version:
            with open(entry["filename"], "rb") as kf:
                key = kf.read()
                return Fernet(key)

    raise ValueError(f"Key version {version} not found.")


In [ ]:
def encrypt_file(input_path, output_path, fernet_cipher):
    with open(input_path, "rb") as f:
        data = f.read()
    encrypted = fernet_cipher.encrypt(data)
    with open(output_path, "wb") as f:
        f.write(encrypted)
    print(f"🔐 Encrypted file saved as {output_path}")

def decrypt_file(encrypted_path, output_path, fernet_cipher):
    with open(encrypted_path, "rb") as f:
        encrypted = f.read()
    decrypted = fernet_cipher.decrypt(encrypted)
    with open(output_path, "wb") as f:
        f.write(decrypted)
    print(f"🔓 Decrypted file saved as {output_path}")


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving IDK-wallpaper.png to IDK-wallpaper.png


In [ ]:
import json
import os
# 📁 Define vault file path
vault_file = "keyvault/vault.json"

# Set version number (based on number of keys in vault)
if not os.path.exists(vault_file):
    with open(vault_file, "w") as f:
        json.dump({"keys": []}, f)

with open(vault_file, "r") as f:
    current_data = json.load(f)

new_version = len(current_data["keys"]) + 1

# Generate and save key with metadata
new_key = generate_key()
save_key_with_metadata(new_key, new_version)

# Load Fernet object for current key version
cipher = load_key(new_version)

# Encrypt file
input_file = list(uploaded.keys())[0]
encrypted_file = f"{input_file}.enc"
encrypt_file(input_file, encrypted_file, cipher)

# Decrypt file
decrypted_file = f"decrypted_{input_file}"
decrypt_file(encrypted_file, decrypted_file, cipher)


✅ Key version 2 saved with metadata.
🔐 Encrypted file saved as IDK-wallpaper.png.enc
🔓 Decrypted file saved as decrypted_IDK-wallpaper.png


In [ ]:
with open(vault_file, "r") as f:
    vault_data = json.load(f)

print("📜 Key Vault Metadata:\n")
for key_entry in vault_data["keys"]:
    print(f"Version: {key_entry['version']} | Saved: {key_entry['timestamp']} | File: {key_entry['filename']}")


📜 Key Vault Metadata:

Version: 1 | Saved: 2025-07-31T16:56:17.751673 | File: keyvault/key_v1.key
Version: 2 | Saved: 2025-07-31T16:56:25.150986 | File: keyvault/key_v2.key
